# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [97]:
%matplotlib inline
%reset -f

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from emoji import UNICODE_EMOJI


data = pd.ExcelFile('tweets_Tinder_201809041724.xlsx')
data_treinamento = pd.read_excel(data, 'Treinamento')
data_teste = pd.read_excel(data, 'Teste')


data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace(',', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('.', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace(':', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('/', ' ')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('#', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('@', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('?', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('?', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('\n', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('rt', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('(', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace(')', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('*', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('“', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('”', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('_', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('&', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace(';', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('%', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace("'", '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('-', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('+', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace("=", '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('"', '')

data_teste.Teste = data_teste.Teste.str.replace(',', '')
data_teste.Teste = data_teste.Teste.str.replace('.', '')
data_teste.Teste = data_teste.Teste.str.replace(':', '')
data_teste.Teste = data_teste.Teste.str.replace('/', ' ')
data_teste.Teste = data_teste.Teste.str.replace('#', '')
data_teste.Teste = data_teste.Teste.str.replace('@', '')
data_teste.Teste = data_teste.Teste.str.replace('?', '')
data_teste.Teste = data_teste.Teste.str.replace('rt', '')
data_teste.Teste = data_teste.Teste.str.replace('\n', '')

lista_espaco = []
for each in data_treinamento['Treinamento']:
    e = ''
    for word in each:
        if word in UNICODE_EMOJI:
            e += " " + word + ' '
        else:
            e += word
    lista_espaco.append(e)
    
data_treinamento['Treinamento'] = lista_espaco

data_treinamento_relevantes = data_treinamento[(data_treinamento['Relevância'] == 1)]
data_treinamento_irrelevantes = data_treinamento[(data_treinamento['Relevância'] == 0)]

data_teste_relevantes = data_teste[(data_teste['Relevância'] == 1)]
data_teste_irrelevantes = data_teste[(data_teste['Relevância'] == 0)]



data_treinamento.head(5)

,Treinamento,Relevância
0,o cara do tinder ficou brabo cmg pq eu disse q...,0
1,fica a questão tinder https tco iq7iyzucdd,0
2,terceira vez que dou match com a mesma pessoa ...,1
3,meta pra 2018 deletar o tinder ainda esse ano ...,0
4,para todos os matches do tinder que eu não man...,0


In [85]:
tweets = []
relevante = []
irrelevante = []

for each in data_treinamento['Treinamento']:
    each = each.lower()
    
for each in data_treinamento_relevantes['Treinamento']:
    relevante.append(each.split())
    
for each in data_treinamento_irrelevantes['Treinamento']:
    irrelevante.append(each.split())

for each in data_treinamento['Treinamento']:
    tweets.append(each.split())


n_irrelevantes = data_treinamento.Relevância[data_treinamento.Relevância == 0].count()
n_relevantes = data_treinamento.Relevância[data_treinamento.Relevância == 1].count()

total = n_relevantes + n_irrelevantes

prob_irrelevante = n_irrelevantes/ total
prob_relevante = n_relevantes/ total

dicionario_rel = {}
dicionario_irrel = {}
dicionario_todos = {}

lista_nova_rel = []
lista_nova_irrel = []
todos = []

for lista in relevante:
    for word in lista:
        lista_nova_rel.append(word)
        dicionario_rel[word] = lista_nova_rel.count(word)

for lista in irrelevante:
    for word in lista:
        lista_nova_irrel.append(word)
        dicionario_irrel[word] = lista_nova_irrel.count(word)

for lista in tweets:
    for word in lista:
        todos.append(word)
        dicionario_todos[word] = todos.count(word)

In [86]:
print(len(todos))
print(len(lista_nova_rel))
print(len(lista_nova_irrel))

5142
1573
3569


In [87]:
a= list(set(todos))
print(len(a))
print(len(todos))

1470
5142


In [90]:
r = len(lista_nova_rel)/len(todos)
ir = len(lista_nova_irrel)/len(todos)

In [91]:
p_r = {}
p_ir = {}

for word in lista_nova_rel:
    p_r[word] = (dicionario_rel[word] + 1)/ (len(a) + len(lista_nova_rel))

for word in lista_nova_irrel:
    p_ir[word] = (dicionario_irrel[word] + 1)/ (len(a) + len(lista_nova_irrel))

In [93]:
p_r

{'17': 0.0006572461386789353,
 '19': 0.0006572461386789353,
 '1ia6dswzqh': 0.0006572461386789353,
 '2014': 0.0006572461386789353,
 '35cwc66axs': 0.0006572461386789353,
 '4aqq8c7qyy': 0.0006572461386789353,
 '7bkezqxl6b': 0.0006572461386789353,
 '8266': 0.0006572461386789353,
 '8fiilvv20i': 0.0006572461386789353,
 '8kkh6dbjhh': 0.0006572461386789353,
 '91xob1yfag': 0.0006572461386789353,
 'Hahahah': 0.0006572461386789353,
 'a': 0.007558330594807755,
 'abaixa': 0.0006572461386789353,
 'acabam': 0.0006572461386789353,
 'acabei': 0.0013144922773578706,
 'acha': 0.0006572461386789353,
 'achei': 0.0013144922773578706,
 'acho': 0.0013144922773578706,
 'acjunior95': 0.0006572461386789353,
 'acontecesse': 0.0006572461386789353,
 'aconteceu': 0.000985869208018403,
 'acordei': 0.0006572461386789353,
 'acredito': 0.0006572461386789353,
 'afastamos': 0.0006572461386789353,
 'afinidade': 0.0026289845547157412,
 'afins': 0.0006572461386789353,
 'agora': 0.0013144922773578706,
 'ai': 0.000657246138678

In [96]:
d = sum(p_r.values())
b = sum(p_ir.values())

print(b + d)

1.624465547304819


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [99]:
resultado = []
data_teste

,Treinamento,Relevância
0,o cara do tinder ficou brabo cmg pq eu disse q...,0
1,fica a questão tinder https tco iq7iyzucdd,0
2,terceira vez que dou match com a mesma pessoa ...,1
3,meta pra 2018 deletar o tinder ainda esse ano ...,0
4,para todos os matches do tinder que eu não man...,0
5,eduaaarda eu n sei pq ainda insisto em instal...,1
6,tal qual o ubero tinder tinha que ter uma área...,1
7,to muito confuso hoje to tipo gente que namora...,0
8,parebruno pensei que esse lgbtqtwiter fosse p...,0
9,como tem bastante gente que não mandou foto po...,0


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
